## 1. Background

Chinese numerology, or bazi, is a common cultural practice used by many Chinese for date selection and character profiling. Every year, month, day and hour is represented by a pillar, which is combination of a Heavenly Stem (base-10) and the Earthly Branch (base-12) character. The respective pillars can be found in the Chinese solar sexagenary calendar. Every stem and branch represents certain elements, and the interaction of all the elements present at a particular time is used to predict the outcomes of events. 

To convert a date from the Gregorian calendar to the Chinese solar sexagenary calendar, a book called *Tung Shing* (通勝) is used. The *Tung Shing* also contains description of the suitable activities of the day, which many fengshui practioners and geomancers use.  

## 2. Problem statement

While there are free websites and apps that provides information for individual dates, the ability to extract multiple dates fulfiling one or more criteria is usually not available, or only found on the paid tier. In this project, we are going to create a digital database containing some basic information of the *Tong Shing*, which can queried and searched using SQL language.

## 3. Dataset

We are going to use the open-source information found on [Wikibooks](https://en.wikibooks.org/wiki/Ba_Zi).

## 4. Import dependencies

In [2]:
import pandas as pd
import requests
import numpy as np
import re

## 5. Web scraping

### 5.1. Scrape birth date table for all available years

#### 5.1.1. Send http request, receive html responses and convert html to list of dataframe objects

In [41]:
%%time
lst_birth = []
for year in range(1902, 2049): #2049
    url = 'https://en.wikibooks.org/wiki/Ba_Zi/Yr' + str(year)
    html = requests.get(url, timeout = 5, verify = True)
    dfs = pd.read_html(html.text)
    for x in dfs:
        lst_birth.append(x)
len(lst_birth)

Wall time: 29 s


1764

In [42]:
#checking results for 2 months 
lst_birth[0:2]

[    S/N  Year  Month  Day  SS  Hour Pillar Day Pillar Month Pillar  \
 0     1  1902      1    1  11   Time of H1      H1_E9        H7_E1   
 1     2  1902      1    2  11   Time of H2     H2_E10        H7_E1   
 2     3  1902      1    3  11   Time of H3     H3_E11        H7_E1   
 3     4  1902      1    4  11   Time of H4     H4_E12        H7_E1   
 4     5  1902      1    5  11   Time of H5      H5_E1        H7_E1   
 5     6  1902      1    6  12   Time of H6      H6_E2        H8_E2   
 6     7  1902      1    7  12   Time of H7      H7_E3        H8_E2   
 7     8  1902      1    8  12   Time of H8      H8_E4        H8_E2   
 8     9  1902      1    9  12   Time of H9      H9_E5        H8_E2   
 9    10  1902      1   10  12  Time of H10     H10_E6        H8_E2   
 10   11  1902      1   11  12   Time of H1      H1_E7        H8_E2   
 11   12  1902      1   12  12   Time of H2      H2_E8        H8_E2   
 12   13  1902      1   13  12   Time of H3      H3_E9        H8_E2   
 13   

#### 5.1.2. Merge all years into single dataframe

In [43]:
df_birth = pd.concat([x for x in lst_birth], axis = 0, ignore_index=True) #stack all 12 months of each year on top of each other
df_birth.head()

,S/N,Year,Month,Day,SS,Hour Pillar,Day Pillar,Month Pillar,Year Pillar,Male Luck Pillar,Female Luck Pillar
0,1,1902,1,1,11,Time of H1,H1_E9,H7_E1,H8_E2,Age 8 (H6E12r),Age 2 (H8E2f)
1,2,1902,1,2,11,Time of H2,H2_E10,H7_E1,H8_E2,Age 9 (H6E12r),Age 1 (H8E2f)
2,3,1902,1,3,11,Time of H3,H3_E11,H7_E1,H8_E2,Age 9 (H6E12r),Age 1 (H8E2f)
3,4,1902,1,4,11,Time of H4,H4_E12,H7_E1,H8_E2,Age 9 (H6E12r),Age 1 (H8E2f)
4,5,1902,1,5,11,Time of H5,H5_E1,H7_E1,H8_E2,Age 10 (H6E12r),Age 0 (H8E2f)


In [44]:
len(df_birth)

53692

#### 5.1.3. Rename columns

In [45]:
df_birth.columns = df_birth.columns.str.lower()
df_birth.columns = df_birth.columns.str.replace(' ', '_')
df_birth.rename(columns={'SS': 'season' }, inplace=True)

df_birth.columns

Index(['s/n', 'year', 'month', 'day', 'ss', 'hour_pillar', 'day_pillar',
       'month_pillar', 'year_pillar', 'male_luck_pillar',
       'female_luck_pillar'],
      dtype='object')

#### 5.1.4 Check for null values

In [46]:
df_birth.isnull().mean().sum()# no null values

0.0

No null values in birth chart dataframe.

#### 5.1.5 Check for data types

In [47]:
df_birth.dtypes

s/n                    int64
year                   int64
month                  int64
day                    int64
ss                     int64
hour_pillar           object
day_pillar            object
month_pillar          object
year_pillar           object
male_luck_pillar      object
female_luck_pillar    object
dtype: object

Date elements and order of season are in integers, while pillars are in object type (string).

#### 5.1.6. Expand year, month and day pillars to obtain stems and branches

To facilitate later SQL queries, we will expand the information that had been compressed into the pillars into new columns.

In [48]:
#expand year pillar
new = df_birth['year_pillar'].str.split('_', expand = True)
df_birth['year_stem'] = new[0]
df_birth['year_branch'] = new[1]

#expand month pillar
new = df_birth['month_pillar'].str.split('_', expand = True)
df_birth['mth_stem'] = new[0]
df_birth['mth_branch'] = new[1]

#expand day pillar
new = df_birth['day_pillar'].str.split('_', expand = True)
df_birth['day_stem'] = new[0]
df_birth['day_branch'] = new[1]

df_birth.head()

,s/n,year,month,day,ss,hour_pillar,day_pillar,month_pillar,year_pillar,male_luck_pillar,female_luck_pillar,year_stem,year_branch,mth_stem,mth_branch,day_stem,day_branch
0,1,1902,1,1,11,Time of H1,H1_E9,H7_E1,H8_E2,Age 8 (H6E12r),Age 2 (H8E2f),H8,E2,H7,E1,H1,E9
1,2,1902,1,2,11,Time of H2,H2_E10,H7_E1,H8_E2,Age 9 (H6E12r),Age 1 (H8E2f),H8,E2,H7,E1,H2,E10
2,3,1902,1,3,11,Time of H3,H3_E11,H7_E1,H8_E2,Age 9 (H6E12r),Age 1 (H8E2f),H8,E2,H7,E1,H3,E11
3,4,1902,1,4,11,Time of H4,H4_E12,H7_E1,H8_E2,Age 9 (H6E12r),Age 1 (H8E2f),H8,E2,H7,E1,H4,E12
4,5,1902,1,5,11,Time of H5,H5_E1,H7_E1,H8_E2,Age 10 (H6E12r),Age 0 (H8E2f),H8,E2,H7,E1,H5,E1


#### 5.1.7. Expand luck pillar

In [49]:
%%time
def find_lp_stem(x):
    stem = re.search('H\d+', x)
    return stem.group()

def find_lp_branch(x):
    branch = re.search('E\d+', x)
    return branch.group()

df_birth['male_lp_age'] = df_birth['male_luck_pillar'].str.split(' ').map(lambda x: x[1])
df_birth['female_lp_age'] = df_birth['female_luck_pillar'].str.split(' ').map(lambda x: x[1])
df_birth['male_lp_dir'] = df_birth['male_luck_pillar'].str.split(' ').map(lambda x: x[2][-2])
df_birth['female_lp_dir'] = df_birth['female_luck_pillar'].str.split(' ').map(lambda x: x[2][-2])
df_birth['male_lp_pillar'] = df_birth['male_luck_pillar'].str.split(' ').map(lambda x: x[2][1:-2])
df_birth['female_lp_pillar'] = df_birth['female_luck_pillar'].str.split(' ').map(lambda x: x[2][1:-2])
df_birth['male_lp_stem'] = df_birth['male_luck_pillar'].map(lambda x: find_lp_stem(x))
df_birth['male_lp_branch'] = df_birth['male_luck_pillar'].map(lambda x: find_lp_branch(x))
df_birth['female_lp_stem'] = df_birth['female_luck_pillar'].map(lambda x: find_lp_stem(x))
df_birth['female_lp_branch'] = df_birth['female_luck_pillar'].map(lambda x: find_lp_branch(x))

# repopulate luck pillar with hour stem and branch
df_birth['male_lp_pillar'] = df_birth['male_lp_stem'] + '_' + df_birth['male_lp_branch']
df_birth['female_lp_pillar'] = df_birth['female_lp_stem'] + '_' + df_birth['female_lp_branch']

df_birth.head()

Wall time: 552 ms


,s/n,year,month,day,ss,hour_pillar,day_pillar,month_pillar,year_pillar,male_luck_pillar,...,male_lp_age,female_lp_age,male_lp_dir,female_lp_dir,male_lp_pillar,female_lp_pillar,male_lp_stem,male_lp_branch,female_lp_stem,female_lp_branch
0,1,1902,1,1,11,Time of H1,H1_E9,H7_E1,H8_E2,Age 8 (H6E12r),...,8,2,r,f,H6_E12,H8_E2,H6,E12,H8,E2
1,2,1902,1,2,11,Time of H2,H2_E10,H7_E1,H8_E2,Age 9 (H6E12r),...,9,1,r,f,H6_E12,H8_E2,H6,E12,H8,E2
2,3,1902,1,3,11,Time of H3,H3_E11,H7_E1,H8_E2,Age 9 (H6E12r),...,9,1,r,f,H6_E12,H8_E2,H6,E12,H8,E2
3,4,1902,1,4,11,Time of H4,H4_E12,H7_E1,H8_E2,Age 9 (H6E12r),...,9,1,r,f,H6_E12,H8_E2,H6,E12,H8,E2
4,5,1902,1,5,11,Time of H5,H5_E1,H7_E1,H8_E2,Age 10 (H6E12r),...,10,0,r,f,H6_E12,H8_E2,H6,E12,H8,E2


#### 5.1.8. Drop irrelevant columns from birth date table

In [50]:
df_birth = df_birth.drop(columns = ['s/n', 'male_luck_pillar','female_luck_pillar', 'hour_pillar'])
# check final columns
df_birth.columns

Index(['year', 'month', 'day', 'ss', 'day_pillar', 'month_pillar',
       'year_pillar', 'year_stem', 'year_branch', 'mth_stem', 'mth_branch',
       'day_stem', 'day_branch', 'male_lp_age', 'female_lp_age', 'male_lp_dir',
       'female_lp_dir', 'male_lp_pillar', 'female_lp_pillar', 'male_lp_stem',
       'male_lp_branch', 'female_lp_stem', 'female_lp_branch'],
      dtype='object')

In [51]:
# Check final form of birth date table
df_birth.head()

,year,month,day,ss,day_pillar,month_pillar,year_pillar,year_stem,year_branch,mth_stem,...,male_lp_age,female_lp_age,male_lp_dir,female_lp_dir,male_lp_pillar,female_lp_pillar,male_lp_stem,male_lp_branch,female_lp_stem,female_lp_branch
0,1902,1,1,11,H1_E9,H7_E1,H8_E2,H8,E2,H7,...,8,2,r,f,H6_E12,H8_E2,H6,E12,H8,E2
1,1902,1,2,11,H2_E10,H7_E1,H8_E2,H8,E2,H7,...,9,1,r,f,H6_E12,H8_E2,H6,E12,H8,E2
2,1902,1,3,11,H3_E11,H7_E1,H8_E2,H8,E2,H7,...,9,1,r,f,H6_E12,H8_E2,H6,E12,H8,E2
3,1902,1,4,11,H4_E12,H7_E1,H8_E2,H8,E2,H7,...,9,1,r,f,H6_E12,H8_E2,H6,E12,H8,E2
4,1902,1,5,11,H5_E1,H7_E1,H8_E2,H8,E2,H7,...,10,0,r,f,H6_E12,H8_E2,H6,E12,H8,E2


### 5.2. Scrape table on hour pillar calculation

#### 5.2.1. Send http request, receive html responses and convert html to dataframe

In [52]:
%%time
url_hour = 'https://en.wikibooks.org/wiki/Ba_Zi/Hour_Pillar'
html = requests.get(url_hour, timeout = 5, verify = True)
df_hour = pd.read_html(html.text)
df_hour = df_hour[0]
df_hour

Wall time: 35.3 ms


,Heavenly Stem of Day Pillar versus Time of Birth(时辰),H1,H2,H3,H4,H5,H6,H7,H8,H9,H10,Earthly Branch ofHour Pillar
0,11pm-1am,H1,H3,H5,H7,H9,H1,H3,H5,H7,H9,E1
1,1am-3am,H2,H4,H6,H8,H10,H2,H4,H6,H8,H10,E2
2,3am-5am,H3,H5,H7,H9,H1,H3,H5,H7,H9,H1,E3
3,5am-7am,H4,H6,H8,H10,H2,H4,H6,H8,H10,H2,E4
4,7am-9am,H5,H7,H9,H1,H3,H5,H7,H9,H1,H3,E5
5,9am-11am,H6,H8,H10,H2,H4,H6,H8,H10,H2,H4,E6
6,11am-1pm,H7,H9,H1,H3,H5,H7,H9,H1,H3,H5,E7
7,1pm-3pm,H8,H10,H2,H4,H6,H8,H10,H2,H4,H6,E8
8,3pm-5pm,H9,H1,H3,H5,H7,H9,H1,H3,H5,H7,E9
9,5pm-7pm,H10,H2,H4,H6,H8,H10,H2,H4,H6,H8,E10


#### 5.2.2. Rename columns

In [53]:
df_hour.rename(columns={'Heavenly Stem of Day Pillar versus Time of Birth(时辰)': 'hour_range', 'Earthly Branch ofHour Pillar': 'hour_branch'}, inplace=True)
df_hour.head()                

,hour_range,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10,hour_branch
0,11pm-1am,H1,H3,H5,H7,H9,H1,H3,H5,H7,H9,E1
1,1am-3am,H2,H4,H6,H8,H10,H2,H4,H6,H8,H10,E2
2,3am-5am,H3,H5,H7,H9,H1,H3,H5,H7,H9,H1,E3
3,5am-7am,H4,H6,H8,H10,H2,H4,H6,H8,H10,H2,E4
4,7am-9am,H5,H7,H9,H1,H3,H5,H7,H9,H1,H3,E5


#### 5.2.3. Add hour_pair column

We will add an hour_pair value in lieu of the hour_range, for ease of locating the correct hour branch when calculating the hour pillar.

In [54]:
df_hour['hour_pair'] = df_hour.index + 1
df_hour.head()  

,hour_range,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10,hour_branch,hour_pair
0,11pm-1am,H1,H3,H5,H7,H9,H1,H3,H5,H7,H9,E1,1
1,1am-3am,H2,H4,H6,H8,H10,H2,H4,H6,H8,H10,E2,2
2,3am-5am,H3,H5,H7,H9,H1,H3,H5,H7,H9,H1,E3,3
3,5am-7am,H4,H6,H8,H10,H2,H4,H6,H8,H10,H2,E4,4
4,7am-9am,H5,H7,H9,H1,H3,H5,H7,H9,H1,H3,E5,5


#### 5.2.4. Drop irrelevant columns

In [55]:
df_hour.drop(columns = 'hour_range', inplace = True)
df_hour.head()

,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10,hour_branch,hour_pair
0,H1,H3,H5,H7,H9,H1,H3,H5,H7,H9,E1,1
1,H2,H4,H6,H8,H10,H2,H4,H6,H8,H10,E2,2
2,H3,H5,H7,H9,H1,H3,H5,H7,H9,H1,E3,3
3,H4,H6,H8,H10,H2,H4,H6,H8,H10,H2,E4,4
4,H5,H7,H9,H1,H3,H5,H7,H9,H1,H3,E5,5


#### 5.2.5. Check data types

In [56]:
df_hour.dtypes

H1             object
H2             object
H3             object
H4             object
H5             object
H6             object
H7             object
H8             object
H9             object
H10            object
hour_branch    object
hour_pair       int64
dtype: object

All stems and branches are object type (string) except for the hour_pair, which is rightly an integer. 

### 5.3. Scrape table on elements in earthly branches

Each Earthly Branch in the four Pillars will have one or more Heavenly Stems.

#### 5.3.1. Send http request, receive html responses and convert html to dataframe

In [57]:
%%time
url_elm = 'https://en.wikibooks.org/wiki/Ba_Zi/Hidden_Stems'
html = requests.get(url_elm, timeout = 5, verify = True)
df_elm = pd.read_html(html.text)

Wall time: 32.8 ms


In [58]:
df_elm = df_elm[0]
df_elm

,Earthly Branchin anyPillar,Hidden Heavenly Stems
0,E3,"H5, H1, H3"
1,E4,H2
2,E5,"H10, H5, H2"
3,E6,"H7, H3, H5"
4,E7,"H4, H6"
5,E8,"H4, H6, H2"
6,E9,"H5, H7, H9"
7,E10,H8
8,E11,"H4, H5, H8"
9,E12,"H9, H1"


#### 5.3.2. Rename columns

In [59]:
df_elm.rename(columns={'Earthly Branchin anyPillar': 'branch', 'Hidden Heavenly Stems': 'all_stems'}, inplace=True)
df_elm.columns

Index(['branch', 'all_stems'], dtype='object')

#### 5.3.3. Define main and hidden stems in branch

In [60]:
#create a new column indicating main stem in branch
def main_elm(x):
    y = x.split(', ')
    z = round(0.5*len(y))
    return y[z-1]

#create a new column indicating hidden stems in branch
def hidden_elm(x):
    y = x.split(', ')
    if len(y) == 1:
        z = np.nan
    elif len(y)==3:
        z = y[0] + ', ' + y[2]
    elif len(y)==2:
        z = y[1]
    return z

df_elm['main_stem'] = df_elm['all_stems'].apply(lambda x: main_elm(x))
df_elm['hidden_stems'] = df_elm['all_stems'].apply(lambda x: hidden_elm(x))
df_elm

,branch,all_stems,main_stem,hidden_stems
0,E3,"H5, H1, H3",H1,"H5, H3"
1,E4,H2,H2,NaN
2,E5,"H10, H5, H2",H5,"H10, H2"
3,E6,"H7, H3, H5",H3,"H7, H5"
4,E7,"H4, H6",H4,H6
5,E8,"H4, H6, H2",H6,"H4, H2"
6,E9,"H5, H7, H9",H7,"H5, H9"
7,E10,H8,H8,NaN
8,E11,"H4, H5, H8",H5,"H4, H8"
9,E12,"H9, H1",H9,H1


### 5.4. Scrape earthly branch association table

#### 5.4.1. Send http request, receive html responses and convert html to dataframe

In [61]:
%%time
url_eb_asc = 'https://en.wikibooks.org/wiki/Ba_Zi/Earthly_Branches'
html = requests.get(url_eb_asc, timeout = 5, verify = True)
df_eb_asc_scraped = pd.read_html(html.text)

Wall time: 39.2 ms


In [62]:
df_eb_asc_scraped

[    S/N    Time(时辰) Notation Chinese Pinyin  \
 0     1  11pm - 1am      EB1             zǐ   
 1     2   1am - 3am      EB2           chǒu   
 2     3   3am - 5am      EB3            yín   
 3     4   5am - 7am      EB4            mǎo   
 4     5   7am - 9am      EB5           chén   
 5     6  9am - 11am      EB6             sì   
 6     7  11am - 1pm      EB7             wǔ   
 7     8   1pm - 3pm      EB8            wèi   
 8     9   3pm - 5pm      EB9           shēn   
 9    10   5pm - 7pm     EB10            yǒu   
 10   11    7pm -9pm     EB11             xū   
 11   12  9pm - 11pm     EB12            hài   
 
    Earthly Branch in Simplified Chinese(地支) 12 Animal Mnemonics(十二支)  \
 0                                         子                      Rat   
 1                                         丑                       Ox   
 2                                         寅                    Tiger   
 3                                         卯                   Rabbit   
 4       

In [63]:
df_eb_asc = df_eb_asc_scraped[0]
df_eb_asc

,S/N,Time(时辰),Notation,Chinese Pinyin,Earthly Branch in Simplified Chinese(地支),12 Animal Mnemonics(十二支),Yin/Yang(阴阳),Five Elements(五行)
0,1,11pm - 1am,EB1,zǐ,子,Rat,Yang,Water
1,2,1am - 3am,EB2,chǒu,丑,Ox,Yin,Earth
2,3,3am - 5am,EB3,yín,寅,Tiger,Yang,Wood
3,4,5am - 7am,EB4,mǎo,卯,Rabbit,Yin,Wood
4,5,7am - 9am,EB5,chén,辰,Dragon,Yang,Earth
5,6,9am - 11am,EB6,sì,巳,Snake,Yin,Fire
6,7,11am - 1pm,EB7,wǔ,午,Horse,Yang,Fire
7,8,1pm - 3pm,EB8,wèi,未,Goat,Yin,Earth
8,9,3pm - 5pm,EB9,shēn,申,Monkey,Yang,Metal
9,10,5pm - 7pm,EB10,yǒu,酉,Rooster,Yin,Metal


#### 5.4.2. Drop columns

In [64]:
df_eb_asc.drop(columns = ['S/N', 'Chinese Pinyin'], inplace = True)
df_eb_asc

,Time(时辰),Notation,Earthly Branch in Simplified Chinese(地支),12 Animal Mnemonics(十二支),Yin/Yang(阴阳),Five Elements(五行)
0,11pm - 1am,EB1,子,Rat,Yang,Water
1,1am - 3am,EB2,丑,Ox,Yin,Earth
2,3am - 5am,EB3,寅,Tiger,Yang,Wood
3,5am - 7am,EB4,卯,Rabbit,Yin,Wood
4,7am - 9am,EB5,辰,Dragon,Yang,Earth
5,9am - 11am,EB6,巳,Snake,Yin,Fire
6,11am - 1pm,EB7,午,Horse,Yang,Fire
7,1pm - 3pm,EB8,未,Goat,Yin,Earth
8,3pm - 5pm,EB9,申,Monkey,Yang,Metal
9,5pm - 7pm,EB10,酉,Rooster,Yin,Metal


#### 5.4.3. Rename columns

In [65]:
df_eb_asc.rename(columns = {
    'Time(时辰)':'time',
    'Notation': 'branch',
    'Earthly Branch in Simplified Chinese(地支)': 'chinese',
    '12 Animal Mnemonics(十二支)': 'animal',
    'Yin/Yang(阴阳)':'ying_yang',
    'Five Elements(五行)': 'element'
}, inplace = True)

df_eb_asc

,time,branch,chinese,animal,ying_yang,element
0,11pm - 1am,EB1,子,Rat,Yang,Water
1,1am - 3am,EB2,丑,Ox,Yin,Earth
2,3am - 5am,EB3,寅,Tiger,Yang,Wood
3,5am - 7am,EB4,卯,Rabbit,Yin,Wood
4,7am - 9am,EB5,辰,Dragon,Yang,Earth
5,9am - 11am,EB6,巳,Snake,Yin,Fire
6,11am - 1pm,EB7,午,Horse,Yang,Fire
7,1pm - 3pm,EB8,未,Goat,Yin,Earth
8,3pm - 5pm,EB9,申,Monkey,Yang,Metal
9,5pm - 7pm,EB10,酉,Rooster,Yin,Metal


#### 5.4.4. Standardise branch notation

In [66]:
df_eb_asc['branch'] = df_eb_asc['branch'].apply(lambda x: re.sub(r'EB','E', x))
df_eb_asc

,time,branch,chinese,animal,ying_yang,element
0,11pm - 1am,E1,子,Rat,Yang,Water
1,1am - 3am,E2,丑,Ox,Yin,Earth
2,3am - 5am,E3,寅,Tiger,Yang,Wood
3,5am - 7am,E4,卯,Rabbit,Yin,Wood
4,7am - 9am,E5,辰,Dragon,Yang,Earth
5,9am - 11am,E6,巳,Snake,Yin,Fire
6,11am - 1pm,E7,午,Horse,Yang,Fire
7,1pm - 3pm,E8,未,Goat,Yin,Earth
8,3pm - 5pm,E9,申,Monkey,Yang,Metal
9,5pm - 7pm,E10,酉,Rooster,Yin,Metal


#### 5.4.5. Merge with earlier branch-related table

In [67]:
left_join = pd.merge(df_elm, 
                      df_eb_asc, 
                      on ='branch', 
                      how ='left')
df_branch = left_join
df_branch.head(15)

,branch,all_stems,main_stem,hidden_stems,time,chinese,animal,ying_yang,element
0,E3,"H5, H1, H3",H1,"H5, H3",3am - 5am,寅,Tiger,Yang,Wood
1,E4,H2,H2,NaN,5am - 7am,卯,Rabbit,Yin,Wood
2,E5,"H10, H5, H2",H5,"H10, H2",7am - 9am,辰,Dragon,Yang,Earth
3,E6,"H7, H3, H5",H3,"H7, H5",9am - 11am,巳,Snake,Yin,Fire
4,E7,"H4, H6",H4,H6,11am - 1pm,午,Horse,Yang,Fire
5,E8,"H4, H6, H2",H6,"H4, H2",1pm - 3pm,未,Goat,Yin,Earth
6,E9,"H5, H7, H9",H7,"H5, H9",3pm - 5pm,申,Monkey,Yang,Metal
7,E10,H8,H8,NaN,5pm - 7pm,酉,Rooster,Yin,Metal
8,E11,"H4, H5, H8",H5,"H4, H8",7pm -9pm,戌,Dog,Yang,Earth
9,E12,"H9, H1",H9,H1,9pm - 11pm,亥,Pig,Yin,Water


### 5.5. Scrape heavenly stem association table

#### 5.5.1. Send http request, receive html responses and convert html to dataframe

In [68]:
%%time
url_hs_asc = 'https://en.wikibooks.org/wiki/Ba_Zi/Heavenly_Stems'
html = requests.get(url_hs_asc, timeout = 5, verify = True)
df_hs_asc_scraped = pd.read_html(html.text)

Wall time: 36 ms


In [69]:
df_hs_asc_scraped

[   S/N Heavenly Stem(天干) Notation Chinese Pinyin Simplified Chinese  \
 0    1               Jia      HS1            jiǎ                  甲   
 1    2                Yi      HS2             yǐ                  乙   
 2    3              Bing      HS3           bǐng                  丙   
 3    4              Ding      HS4           dīng                  丁   
 4    5                Wu      HS5             wù                  戊   
 5    6                Ji      HS6             jǐ                  己   
 6    7              Geng      HS7           gēng                  庚   
 7    8               Xin      HS8            xīn                  辛   
 8    9               Ren      HS9            rén                  壬   
 9   10               Gui     HS10            guǐ                  癸   
 
   Yin/Yang(阴阳) Five Elements(五行)   Zang/Fu(organ)  
 0         Yang              Wood     Gall Bladder  
 1          Yin              Wood            Liver  
 2         Yang              Fire  Small Intest

In [70]:
df_stem = df_hs_asc_scraped[0]
df_stem

,S/N,Heavenly Stem(天干),Notation,Chinese Pinyin,Simplified Chinese,Yin/Yang(阴阳),Five Elements(五行),Zang/Fu(organ)
0,1,Jia,HS1,jiǎ,甲,Yang,Wood,Gall Bladder
1,2,Yi,HS2,yǐ,乙,Yin,Wood,Liver
2,3,Bing,HS3,bǐng,丙,Yang,Fire,Small Intestine
3,4,Ding,HS4,dīng,丁,Yin,Fire,Heart
4,5,Wu,HS5,wù,戊,Yang,Earth,Stomach
5,6,Ji,HS6,jǐ,己,Yin,Earth,Spleen
6,7,Geng,HS7,gēng,庚,Yang,Metal,Large Intestine
7,8,Xin,HS8,xīn,辛,Yin,Metal,Lung
8,9,Ren,HS9,rén,壬,Yang,Water,Urinary Bladder
9,10,Gui,HS10,guǐ,癸,Yin,Water,Kidney


#### 5.5.2. Drop columns

In [71]:
df_stem = df_stem.drop(columns = ['S/N', 'Chinese Pinyin'])
df_stem

,Heavenly Stem(天干),Notation,Simplified Chinese,Yin/Yang(阴阳),Five Elements(五行),Zang/Fu(organ)
0,Jia,HS1,甲,Yang,Wood,Gall Bladder
1,Yi,HS2,乙,Yin,Wood,Liver
2,Bing,HS3,丙,Yang,Fire,Small Intestine
3,Ding,HS4,丁,Yin,Fire,Heart
4,Wu,HS5,戊,Yang,Earth,Stomach
5,Ji,HS6,己,Yin,Earth,Spleen
6,Geng,HS7,庚,Yang,Metal,Large Intestine
7,Xin,HS8,辛,Yin,Metal,Lung
8,Ren,HS9,壬,Yang,Water,Urinary Bladder
9,Gui,HS10,癸,Yin,Water,Kidney


#### 5.5.3. Rename columns

In [72]:
df_stem = df_stem.rename(columns = {
    'Heavenly Stem(天干)':'name',
    'Notation': 'stem',
    'Simplified Chinese': 'chinese',
    'Zang/Fu(organ)': 'organ',
    'Yin/Yang(阴阳)':'ying_yang',
    'Five Elements(五行)': 'element'
})

df_stem

,name,stem,chinese,ying_yang,element,organ
0,Jia,HS1,甲,Yang,Wood,Gall Bladder
1,Yi,HS2,乙,Yin,Wood,Liver
2,Bing,HS3,丙,Yang,Fire,Small Intestine
3,Ding,HS4,丁,Yin,Fire,Heart
4,Wu,HS5,戊,Yang,Earth,Stomach
5,Ji,HS6,己,Yin,Earth,Spleen
6,Geng,HS7,庚,Yang,Metal,Large Intestine
7,Xin,HS8,辛,Yin,Metal,Lung
8,Ren,HS9,壬,Yang,Water,Urinary Bladder
9,Gui,HS10,癸,Yin,Water,Kidney


#### 5.5.4. Standardise stem notation

In [73]:
df_stem['stem'] = df_stem['stem'].apply(lambda x: re.sub(r'HS','H', x))
df_stem

,name,stem,chinese,ying_yang,element,organ
0,Jia,H1,甲,Yang,Wood,Gall Bladder
1,Yi,H2,乙,Yin,Wood,Liver
2,Bing,H3,丙,Yang,Fire,Small Intestine
3,Ding,H4,丁,Yin,Fire,Heart
4,Wu,H5,戊,Yang,Earth,Stomach
5,Ji,H6,己,Yin,Earth,Spleen
6,Geng,H7,庚,Yang,Metal,Large Intestine
7,Xin,H8,辛,Yin,Metal,Lung
8,Ren,H9,壬,Yang,Water,Urinary Bladder
9,Gui,H10,癸,Yin,Water,Kidney


## 6. Export cleaned dataframes to csv

In [75]:
df_birth.to_csv('data/df_birth.csv', index = False)
df_hour.to_csv('data/df_hour.csv', index = False)
df_branch.to_csv('data/df_branch.csv', index = False)
df_stem.to_csv('data/df_stem.csv', index = False)

End of Part 1 - We will use the csv files to create a SQL database and run queries using SQL language in PostgreSQL.